# Transfomer from scratch

In [ ]:
! pip install tokenizers transformers
!pip install tqdm
! pip install bertviz
from tokenizers import ByteLevelBPETokenizer

from tokenizers.trainers import BpeTrainer
from tokenizers.models import BPE
from tqdm.notebook import tqdm
from transformers import RobertaTokenizer
from sklearn.model_selection import train_test_split
import os 
import torch.nn as nn
from torch.optim import AdamW

from torchtext.data.metrics import bleu_score

from transformers.optimization import get_linear_schedule_with_warmup

import torch 

from torch.nn import functional as F



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import torch 

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/dataset/language.csv')

In [ ]:
data['English'].to_csv("english.txt",index=False)

In [ ]:
tokenizer_english=ByteLevelBPETokenizer()

In [ ]:
tokenizer_english.train(['english.txt'],vocab_size=50000,min_frequency=2,special_tokens=['<s>','<pad>','</s>',',<unk>','<mask>'])

In [ ]:
tokenizer_english.encode("Hi how are you")

Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
data['Telugu'].to_csv("telugu.txt",index=False)

In [ ]:
tokenizer_telugu=ByteLevelBPETokenizer()

In [ ]:
tokenizer_telugu.train(['telugu.txt'],vocab_size=50000,min_frequency=3,special_tokens=['<s>','<pad>','</s>',',<unk>','<mask>'])

# Position embedding and embedding 

In [ ]:
import torch
import  torch.nn as nn
import tensorflow as tf
import  pandas as pd
import numpy as np
import math



class Embedding(nn.Module):
    def __init__(self,vocab_size,embed_dim):
        super(Embedding,self).__init__()
        self.embedd=nn.Embedding(vocab_size,embed_dim)
     

    def forward(self,x):

        out=self.embedd(x)
        return out

class PositionEmbedding(nn.Module):
    def __init__(self,max_sen=5000,dim_data=512):
        super(PositionEmbedding,self).__init__()
        self.dim=dim_data

        pos=torch.zeros(max_sen,dim_data)

        for x in range(max_sen):
            for y in range(0,dim_data,2):

                pos[x,y]=math.sin(x/(1000**(2*y/self.dim)))
                pos[x,y+1]=math.cos(x/(1000**((2*(y+1))/self.dim)))


        pos=torch.unsqueeze(pos,0)
        self.register_buffer('pe',pos)

    def forward(self,x):

        # The reason we increase the embedding values before addition is to make the positional encoding relatively smaller. This means the original meaning in the embedding vector won’t be lost when we add them together.

        x=x*math.sqrt(self.dim)
        sequnence_lenght=x.size(1) # Batch_size , sequence_length , dimensions
  

        output=x+torch.autograd.Variable(self.pe[:,:sequnence_lenght],requires_grad=False)

        return output


# Multi head attention block

In [ ]:
import math

import  torch.nn as nn
import  torch
import  torch.nn.functional as F
import tensorflow as tf
import numpy as np
import  pandas as pd

class  MultiHeadattention(nn.Module):
    def __init__(self,n_heads=8,dim=512):

        super(MultiHeadattention,self).__init__()
        self.size_of_matrix=int(dim/n_heads)
        self.n_heads=n_heads
        self.key=nn.Linear(dim,dim,bias=False)
        self.query=nn.Linear(dim,dim,bias=False)
        self.value=nn.Linear(dim,dim,bias=False)
        self.out=nn.Linear(n_heads*self.size_of_matrix,dim)
        self.droput=nn.Dropout(0.3)

    def forward(self,key,value,query,mask=None):
      

    

       
      
        batch_size=key.size(0)
        seq_lenght=key.size(1)

        batch_size_query=query.size(0)
        batch_lenght_query=query.size(1)

        key=self.key(key)
        value=self.value(value)
        query=self.query(query)


        
        


        key=key.view(batch_size,seq_lenght,self.n_heads,self.size_of_matrix)

        value =value.view(batch_size, seq_lenght, self.n_heads, self.size_of_matrix)

        query = query.view( batch_size_query, batch_lenght_query, self.n_heads, self.size_of_matrix)





        # key=self.key(key) #
        # query=self.query(query)
        # value=self.value(value)

     

        key=key.transpose(1,2) # Transpose will convert it into batch_size , n_heads , seq_length , size_of_matrix
        query=query.transpose(1,2)
        value=value.transpose(1,2)

        # convert the query matrix into the batch_size , n_heads, size_of_matrix , seq_lenght

      
        scalar_output=torch.matmul(query,key.transpose(-2,-1))

        scalar_output_square_root=scalar_output/math.sqrt(self.size_of_matrix)
      

        

    
        if mask is not None:
            mask=mask.unsqueeze(1)
   
           
            scalar_output_square_root=scalar_output_square_root.masked_fill(mask==0,float('-inf'))  


        output_from_softmax=F.softmax(scalar_output_square_root,dim=-1)


        mutliplication_with_value=torch.matmul(self.droput(output_from_softmax),value)

        

        concat=mutliplication_with_value.transpose(1,2).contiguous().view(batch_size, -1 ,self.n_heads*self.size_of_matrix)

        outout_from_multihead=self.out(concat)

        return outout_from_multihead , output_from_softmax

# EncoderBlock

In [ ]:
class Encoderlayer(nn.Module):
    def __init__(self,dim,expansion_factor,num_heads=8):

        super(Encoderlayer,self).__init__()
        self.attn=MultiHeadattention(num_heads,dim)
        self.droput=nn.Dropout(0.2) 
        self.Layernorm1=nn.LayerNorm(dim)
        self.Layernorm2=nn.LayerNorm(dim)
        self.feedwordnetwork=nn.Sequential(
            nn.Linear( dim, expansion_factor*dim),
            nn.ReLU(),
            nn.Linear(dim*expansion_factor,dim)
        )

    def forward(self,x,mask=None):
       
        x1,att_prob=self.attn(x,x,x,mask)
        x=self.Layernorm1(self.droput(x1)+x)
        x1=self.feedwordnetwork(x)
        x=self.Layernorm2(self.droput(x1)+x)
        return x,att_prob


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class TransformerEncoderBlock(nn.Module):
    def __init__(self,src_lenght,expansion_factor,dim=512,num_heads=8,max_sequence_lenght=5000):

        super(TransformerEncoderBlock,self).__init__()

        self.Embedding=Embedding(src_lenght,dim)
        self.num_layers=6
        self.positionEmbedding=PositionEmbedding(max_sen=max_sequence_lenght,dim_data=dim)
        self.layers = nn.ModuleList([
            Encoderlayer(dim,expansion_factor)
            for _ in range(self.num_layers)
        ])
    
    def forward(self,x,mask):
        embeded_out = self.Embedding(x)
        
        x= self.positionEmbedding(embeded_out)
 

        for layer in self.layers:
            x,attn_prob=layer(x,mask)
            
            
        return x,attn_prob

# DecoderBlock

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,expansion_factor,dim=512,num_heads=8):
        super(DecoderLayer,self).__init__()
        self.attn=MultiHeadattention(num_heads,dim)
        self.attn2=MultiHeadattention(num_heads,dim)
        self.n_layers=6
        self.dropout_1=nn.Dropout(0.2)
        self.Layer_1=nn.LayerNorm(dim)
       
        self.feedforward_network=nn.Sequential(

            nn.Linear(dim,dim*expansion_factor),
            nn.ReLU(),
            nn.Linear(dim*expansion_factor , dim
                      )
        )

    def forward(self,x,encoder_out,target_mask,src_mask):

        x1,_=self.attn(x,x,x,target_mask)
            # picking value from encoder outpout

        x2=self.Layer_1(self.dropout_1(x1)+x)

        x,attn_prob= self.attn2(encoder_out,encoder_out,x2,src_mask)

        x = self.Layer_1(self.dropout_1(x2) + x)

        x1=self.feedforward_network(x)
        x=self.Layer_1(self.dropout_1(x1)+x)

        return x,attn_prob


In [ ]:
import  torch
import torch.nn as nn


class TransformerDecoderBlock(nn.Module):
    def __init__(self,target_vocab_size,expansion_factor,dim=512,max_sequence_length=5000,num_heads=8):
        super(TransformerDecoderBlock,self).__init__()
        self.embedding=Embedding(target_vocab_size,dim)
        self.position_embedding=PositionEmbedding(max_sen=max_sequence_length,dim_data=dim)
        self.dropout_1=nn.Dropout(0.2)

        self.layers = nn.ModuleList([
            DecoderLayer(expansion_factor)
            for _ in range(6)
        ])

    def forward(self,x,encoder_output,masked_target,src_mask):
      
        embedded_vector=self.embedding(x)
        position=self.position_embedding(embedded_vector)
        x=self.dropout_1(position)

        for layer in self.layers:
            x,attn_prob=layer(x,encoder_output,masked_target,src_mask)

        return x,attn_prob     

# Transformer 

In [ ]:
import  torch
import torch.nn as nn


class TransformerModel(nn.Module):
    def __init__(self,vocab_size_source,vocab_size_target,dim,expansion_factor,DEVICE):
        
        super(TransformerModel, self).__init__()
        
        self.encoder=TransformerEncoderBlock(vocab_size_source,expansion_factor)
       
        self.decoder=TransformerDecoderBlock(vocab_size_target,expansion_factor)

        self.logits=nn.Linear(dim,vocab_size_target)

    def mask_target(self,target):
        batch_size , seq_lenght = target.size(0),target.size(1)

        masked_target = torch.tril(torch.ones(seq_lenght,seq_lenght)).expand(1,seq_lenght,seq_lenght)

        # return masked_target.to(DEVICE,dtype=torch.long)
        return  masked_target.to(DEVICE,dtype=torch.int64)

    def pad_mask(self,x,idx=1) :

        # This masking helps not to consider the indexes for training the padded values 

       mask=(x!=1).unsqueeze(-2)

       return mask.to(DEVICE)

        

    def forward(self,x,y):
        # source mask 
        src_mask=self.pad_mask(x).to(DEVICE)
       
        encoder_output,_=self.encoder(x,src_mask)
      
        masked_target= torch.bitwise_and(self.pad_mask(y) ,self.mask_target(y) )

      
   
     
        decoder_output,attent_prob=self.decoder(y,encoder_output,masked_target,src_mask)
       
        output=self.logits(decoder_output)
        probs_output=F.log_softmax(output,-1)

        return probs_output
        


# Creating English & Telugu dataset 

In [ ]:
class EnglishtoTeluguDataset:
    def __init__(self,english,telugu,tokenizer_te,tokenizer_en,max_length=60):
        self.english=english
        self.telugu=telugu
        self.max_len_en=max_length-1
        self.max_len_te=max_length 
        self.tokenizer_en=tokenizer_en
        self.tokenizer_te=tokenizer_te
    def __len__(self):
        return len(self.english)
    def __getitem__(self,item):
        english = self.tokenizer_en.encode(self.english[item]).ids
        telugu  = self.tokenizer_te.encode(self.telugu[item]).ids

        # english adding start ane end of the sentnece 
     
        if len(english) > self.max_len_en-2:
          english=[0]+english[:self.max_len_en-2]+[2]
        else:
          english=[0]+english+[2]

        if len(english) < self.max_len_en:
          english =english + [1]*(self.max_len_en-len(english))
 


        if len(telugu) > self.max_len_te-2:
          telugu=[0]+telugu[:self.max_len_te-2]+[2]
        else:
          telugu=[0]+telugu+[2]

        if len(telugu) < self.max_len_te:
          telugu = telugu + [1]*(self.max_len_te-len(telugu))
   




      

        return {

            "src":torch.tensor(english,dtype=torch.long),
            "targted":torch.tensor(telugu,dtype=torch.long)


        }   
      
        

In [ ]:
def loss_fn(output,target):
    loss_cal=nn.CrossEntropyLoss(ignore_index=1)
    loss=loss_cal(output,target)

    return loss

In [ ]:
def covert_id_to_text(ids,tokenize,eos_indx):

    if ids.dim()==1:
      
        output_tokens=[]
        for token_ids in ids:
            data_list=[]
            data_list.append(token_ids)
         
          
            if int(token_ids)==eos_indx:
                break
            else:
              output_tokens.append(tokenize.decode(data_list).strip())       
        return output_tokens        
    elif ids.dim()==2 :
        return [covert_id_to_text(ids[i, :], tokenize,eos_indx) for i in range(ids.size(0))]

    raise RuntimeError(f' The dimesion should be 2 but we received {ids.dim()} ')    


In [ ]:
actul=[]
pred=[]
import pickle 
def eval_fn(val_data, model):
    model.eval()
    total_loss=0
    steps=0
    hypotheses = []
    references = []
    with torch.no_grad():
        tk = tqdm(val_data, total=len(val_data)) 
        for batch_size , data in enumerate(tk):
            src=data['src']
            target=data['targted']

            src=src.to(DEVICE, dtype=torch.long)
            target=target.to(DEVICE,dtype=torch.long)

            output=model(src,target[:,:-1])
            
            
            loss=loss_fn(output.view(-1,output.size(-1)),target[:,1:].contiguous().view(-1))

            total_loss+=loss.item()

            steps=+1

            output = output.argmax(dim=-1)

            target=target[:,1:]

            # Calculating belu score 
            pred_tokens=covert_id_to_text(output,tokenizer_telugu,2)

            actual_tokens=covert_id_to_text(target,tokenizer_telugu,2)


            hypotheses += pred_tokens
         
       
            references += [[token] for token in  actual_tokens]

  
    actul.append(actual_tokens)
    pred.append(pred_tokens)
    perplexity = np.log(total_loss / len(val_data))
  
    bleu_data = bleu_score(hypotheses, references)
      

    return bleu_data,perplexity
            

In [ ]:
def train_fn(train_data,model,optimizer,clip=1.0,scheduler=None):
    model.train()
    total_loss=0
    step=0
    tk = tqdm(train_data, total=len(train_data)) 

    for batch_size , train_data in enumerate(tk):

       
        src=train_data['src']
        target=train_data['targted']

        src=src.to(DEVICE, dtype=torch.long)
        target=target.to(DEVICE,dtype=torch.long)

       # forward pass 
        optimizer.zero_grad()
        
        output=model(src,target[:,:-1])

       
        
        loss=loss_fn(output.view(-1,output.size(-1)),target[:,1:].contiguous().view(-1)) # 



        total_loss+=loss.item()

        steps=+1

        output = output.argmax(dim=-1)

        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        # tdqm.set_postfix(loss=total_loss/steps)



        perplexity = np.log(total_loss / len(train_data))



    return output ,perplexity


In [ ]:
def run():

    df_train , df_valid= train_test_split(data,test_size=0.3, random_state=42)
    df_train=df_train.reset_index(drop=True)
    df_valid=df_valid.reset_index(drop=True)

    train_data=EnglishtoTeluguDataset(
    english=df_train.English.values,
    telugu=df_train.Telugu.values,
    tokenizer_te=tokenizer_telugu,
    tokenizer_en=tokenizer_english
    )

    

    train_data_loader=torch.utils.data.DataLoader(
    train_data,
    batch_size=128
     )
    


    val_data=EnglishtoTeluguDataset(
     english=df_valid.English.values,
     telugu=df_valid.Telugu.values,
    tokenizer_te=tokenizer_telugu,
    tokenizer_en=tokenizer_english
    

     )
  
    validation_data_loader=torch.utils.data.DataLoader(
    val_data,
    batch_size=128
    )

  

    num_train_steps = int(len(train_data) /128)* 15

    param_optimizer = list(model.parameters())

    # print(param_optimizer)
    
  
    optimizer = AdamW( param_optimizer, lr=1e-4)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    best_bleu4 = float('-inf')
    es_patience = 3
    patience = 0 
    model_path = '/content/drive/MyDrive/Model/model.pth'

    for i in range(15):
        print("Epoch {}/{}".format(i+1,15))

        _ , train_prexlity=train_fn(train_data_loader,model,optimizer,clip=1.0,scheduler=scheduler)
        blue_score,test_prexlity = eval_fn( validation_data_loader,model)

  
    
      
        print(f'Epoch :{i+1} and  train prexlity {train_prexlity:.4f}')

        print(f'Epoch :{i+1} and  test prexlity {test_prexlity:.4f} and belu score on validation is {blue_score}')

        is_best = blue_score > best_bleu4
        if is_best:
            print(f'BLEU score improved ({best_bleu4:.4f} -> {blue_score:.4f}). Saving Model!')
            best_bleu4 = blue_score
            patience = 0
            torch.save(model.state_dict(), model_path)
        else:
            patience += 1
            print(f'Early stopping counter: {patience} out of {es_patience}')
            if patience == es_patience:
                print(f'Early stopping! Best BLEU4: {best_bleu4:.4f}')
                break
       
       



In [ ]:
run()


# Inference

In [ ]:

INPUT_SIZE=tokenizer_english.get_vocab_size()
OUTPUT_SIZE=tokenizer_telugu.get_vocab_size()


DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model=TransformerModel(INPUT_SIZE,OUTPUT_SIZE,512,3,DEVICE)




model.load_state_dict(torch.load('/content/drive/MyDrive/Model/model.pth',map_location=DEVICE))
# reloading the model 

model.to(DEVICE)


In [ ]:
df_tr , df_val = train_test_split(data,test_size=0.3, random_state=42)
df_tr=df_tr.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

train_data=EnglishtoTeluguDataset(
    english=df_tr.English.values,
    telugu=df_tr.Telugu.values,
   tokenizer_te=tokenizer_telugu,
    tokenizer_en=tokenizer_english
    )

train_data_loader=torch.utils.data.DataLoader(
    train_data,
    batch_size=256
     )

val_data=EnglishtoTeluguDataset(
     english=df_val.English.values,
     telugu=df_val.Telugu.values,
   tokenizer_te=tokenizer_telugu,
    tokenizer_en=tokenizer_english
    
    )
validation_data_loader=torch.utils.data.DataLoader(
    val_data,
    batch_size=256
    )


In [ ]:
def testing_function(model,src,DEVICE):
    model.eval()

    target_ids=[]

   
    
    with torch.no_grad():
        src_mask=model.pad_mask(src).to(DEVICE)
        src=src.unsqueeze(0).to(DEVICE)
          
        # calling encoder to get encoder_output
        # get paded mask for input
      
        with torch.no_grad():
            encoder_output,encoder_attention=model.encoder(src,src_mask)

        target_ids=[0]

        for x in range(60):
             target_id=None
            
             target = torch.tensor(target_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)# (1, target_seq_len)
          
             masked_target= torch.bitwise_and(model.pad_mask(target) ,model.mask_target(target)) 

             with torch.no_grad():
                 decoder_output,attn=model.decoder(target,encoder_output,masked_target,src_mask)
                 output_logits=model.logits(decoder_output)
                
                 output=F.log_softmax(output_logits,dim=-1)

                 target_id = output.argmax(dim=-1)[:, -1].item()
               
                
                 target_ids.append(target_id)
                 if target_id==2:
                   break
                  
        return   target_ids ,attn.squeeze(0).cpu().detach().numpy() , encoder_attention.squeeze(0).cpu().detach().numpy()

    

In [ ]:
# 15 
src=val_data.__getitem__(157)['src']
target_data=val_data.__getitem__(157)['targted']

tokenizer_telugu.decode(target_data.cpu().detach().numpy())

'హైదరాబాదు నగరంలో 2003లో మల్టీ మోడల్ రవాణా వ్యవస్థను మొదలు పెట్టారు.\n'

In [ ]:
tokenizer_english.decode(src.cpu().detach().numpy())

'Multi model transportation system was started in Hyderabad in 2003.\n'

In [ ]:
output,decoder_attent,encoder_atten=testing_function(model,src,DEVICE)

In [ ]:
target=[]
for x in output:

  data_list=[]
  data_list.append(x)
  target.append(tokenizer_telugu.decode(data_list))

In [ ]:
print( f' Input to transformer : {tokenizer_english.decode(src.cpu().detach().numpy())}')
print(f' Actual target : {tokenizer_telugu.decode(target_data.cpu().detach().numpy())}')
print(f'Predicted target : {tokenizer_telugu.decode(output)}')

 Input to transformer : Multi model transportation system was started in Hyderabad in 2003.

 Actual target : హైదరాబాదు నగరంలో 2003లో మల్టీ మోడల్ రవాణా వ్యవస్థను మొదలు పెట్టారు.

Predicted target : హైదరాబాదు నగరంలో 2003లో 2003లో మోడల్ రవాణా వ్యవస్థ పెట్టు మొదలైంది.

